# Pull And Run Docker Image
## Using Triton and Bria-attribution-model

In [ ]:
import requests

api_token = '<API_TOKEN>'

def get_repository_access_password():
    """Fetch ECR credentials from localhost endpoint"""
    try:
        headers = {"api_token":  api_token}
        url ="https://engine.prod.bria-api.com"
        response = requests.get(f"{url}/v2/image/attribution/embedding_repository_access", headers=headers)
        response.raise_for_status()
        result = response.json()["result"]
        password = result["password"]
        
        print(f"Successfully fetched ECR credentials, expires at: {result['expiration']}")
        return password
    except Exception as e:
        print(f"Failed to fetch ECR credentials: {e}")
        raise

password = get_repository_access_password()
! echo "$password" | docker login --username AWS --password-stdin 300465780738.dkr.ecr.us-east-1.amazonaws.com

! docker pull 300465780738.dkr.ecr.us-east-1.amazonaws.com/bria-attribution-embeddings:latest
! docker kill $(docker ps -q --filter expose=8002)
! docker run -d   --runtime=nvidia   --shm-size=16gb   -p 8000:8000   -p 8001:8001   -p 8002:8002 300465780738.dkr.ecr.us-east-1.amazonaws.com/bria-attribution-embeddings:latest

# Client Side

## Installations

In [ ]:
import os
import uuid
import requests
import tritonclient.http as httpclient
from PIL import Image
from embedder import BRIAEmbedder

## Inference Request

In [ ]:
url = '<BRIA_EMBEDDINGS_URL>:8000'
image_path = '<IMAGE_PATH>'

triton_client = httpclient.InferenceServerClient(
    url=url,
    # ssl=True,
    # ssl_context_factory=gevent.ssl._create_default_https_context,
)
image = Image.open(image_path)
bria_pipeline = BRIAEmbedder(triton_client)
img_embeddings = bria_pipeline.run_on_image(image)


## Send Attribution API

In [ ]:
model_version = os.environ.get("model_version", "2.3")
attribution_endpoint = os.environ.get("attribution_endpoint", "httsp://engine.prod.bria-api.com/v2/image/attribution/by_embedding")
api_token = '<API_TOKEN>'

try:
    print(f"Read data from queue")
    embeddings_uid = str(uuid.uuid4())
    print(f"embeddings_uid: {embeddings_uid}")
    request_body = {
        "embeddings_base64": img_embeddings,
        "embeddings_uid": embeddings_uid,
        "model_version": 2.3,
        "agent":  api_token,
        "model_name": 'replace-backgroud',
    }
    
    headers = {
        "api_token":  api_token
    }

    if attribution_endpoint:
        response = requests.post(attribution_endpoint, json=request_body, headers=headers)
        if response.status_code != 202 or response.status_code != 200:
            print(f"API request failed with status code {response.status_code}: {response.text}")
            raise Exception(f"API request failed with status code {response.status_code}: {response.text}")
        else:
            print(f"Request body: {request_body}")
except Exception as ex:
    print(f"Error processing record: {ex}")
    raise